In [1]:
from glob import glob
import re

import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output

from langchain.document_loaders import DirectoryLoader
from langchain.schema import Document
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /opt/conda did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


In [2]:
import torch

In [3]:
print("""
This chapter has presented a wide range of material, including some quite complex ideas from philosophy. Some of the discussion may seem to be a long way from the day-to-day
reality of the therapist or the client. It is certainly not essential to have a fluent grasp of all of these ideas in order to understand the therapeutic work which is presented in the case studies which follow. However, the ability to remain open- minded, and to experiment with new approaches and ideas, is an essential feature of CBT. Furthermore, CBT formulation is a process of understanding the connections between various aspects of the experience of the client – including wider social and cultural issues, which influence therapists as much as clients. We hope that the ideas within this chapter will spark the curiosity of our readers, and encourage a lively and responsive approach to therapy.
""")


This chapter has presented a wide range of material, including some quite complex ideas from philosophy. Some of the discussion may seem to be a long way from the day-to-day
reality of the therapist or the client. It is certainly not essential to have a fluent grasp of all of these ideas in order to understand the therapeutic work which is presented in the case studies which follow. However, the ability to remain open- minded, and to experiment with new approaches and ideas, is an essential feature of CBT. Furthermore, CBT formulation is a process of understanding the connections between various aspects of the experience of the client – including wider social and cultural issues, which influence therapists as much as clients. We hope that the ideas within this chapter will spark the curiosity of our readers, and encourage a lively and responsive approach to therapy.



In [4]:
# print("Loading transformer embeddings...")
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

print("Loading LLaMa tokenizer")
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-13b-hf")

print("Loading LLaMa model")
model1 = LlamaForCausalLM.from_pretrained("decapoda-research/llama-13b-hf", load_in_8bit=True, device_map="auto", torch_dtype=torch.float16)
model2 = PeftModel.from_pretrained(model1, "kmnis/ZenAI", torch_dtype=torch.float16)

Loading LLaMa tokenizer


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading LLaMa model


Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [9]:
model2 = PeftModel.from_pretrained(model1, "../train/experiments2/", torch_dtype=torch.float16)

In [10]:
model2.eval()
model2 = torch.compile(model2)

In [11]:
def get_results(question):
    prompt = f"""This chapter has presented a wide range of material, including some quite complex ideas from philosophy. Some of the discussion may seem to be a long way from the day-to-day reality of the therapist or the client. It is certainly not essential to have a fluent grasp of all of these ideas in order to understand the therapeutic work which is presented in the case studies which follow. However, the ability to remain open- minded, and to experiment with new approaches and ideas, is an essential feature of CBT. Furthermore,
"""
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(temperature=0.6, top_p=0.95, repetition_penalty=1.15)

    generation_output = model2.generate(input_ids=input_ids, generation_config=generation_config,
                                       return_dict_in_generate=True, output_scores=False, max_new_tokens=600)

    for out in generation_output.sequences:
        out = tokenizer.decode(out)
        # out = out.split("### Answer:")[1].split("</s>")[0].strip()
        print(out + "\n")

In [12]:
get_results("")

 ⁇  This chapter has presented a wide range of material, including some quite complex ideas from philosophy. Some of the discussion may seem to be a long way from the day-to-day reality of the therapist or the client. It is certainly not essential to have a fluent grasp of all of these ideas in order to understand the therapeutic work which is presented in the case studies which follow. However, the ability to remain open- minded, and to experiment with new approaches and ideas, is an essential feature of CBT. Furthermore,
CBT is a very practical approach, and it is important that practitioners are able to apply their knowledge to real life situations. The following chapters will present a number of different cases, each of which illustrates one aspect of CBT practice. These examples should help you to see how the theory can be applied in everyday clinical settings.
10.2 Summary In this chapter we have looked at the nature of cognitive distortions and how they can affect our thinking a

In [18]:
question = """
Hey, I am Joe and am struggling with an eating disorder.
"""
get_results(question)

 ⁇  This chapter has presented a wide range of material, including some quite complex ideas from philosophy. Some of the discussion may seem to be a long way from the day-to-day reality of the therapist or the client. It is certainly not essential to have a fluent grasp of all of these ideas in order to understand the therapeutic work which is presented in the case studies which follow. However, the ability to remain open- minded, and to experiment with new approaches and ideas, is an essential feature of CBT. Furthermore,
the ability to reflect on one’s own practice, and to consider alternative ways of working, are also important features of good clinical practice (see Chapter 10). The next two chapters will present a series of case examples illustrating how different aspects of cognitive theory can be applied to help people overcome their problems. These case examples will illustrate many of the points made in this chapter, but they will also introduce you to some of the key concepts